In [3]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
from openpyxl import load_workbook
import random
from difflib import SequenceMatcher

# Инициализация токенизатора и модели BERT
model_path = 'bert-Roman'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForQuestionAnswering.from_pretrained(model_path)
model = model.to(torch.device('cpu'))

# Функция для обработки вопроса и получения ответа
def get_answer(question, context):
    inputs = tokenizer(question, context, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    outputs = model(input_ids, attention_mask=attention_mask)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    start_index = start_index.item()
    end_index = end_index.item()

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index+1]))

    return answer

# Загрузка данных из файла Excel
workbook = load_workbook('questions.xlsx')
sheet = workbook.active

# Выбор случайной строки с вопросом
random_row = random.choice(list(sheet.iter_rows(min_row=2, max_row=sheet.max_row, values_only=True)))

# Вывод вопроса
print('Вопрос:', random_row[0])

# Ввод ответа респондента
response = input('Ответ: ')

# Обработка ответа и получение предсказанного ответа
predicted_answer = get_answer(random_row[0], response)

# Сравнение предсказанного ответа с ожидаемым ответом с использованием SequenceMatcher
expected_answer = random_row[1]
similarity_ratio = SequenceMatcher(None, predicted_answer.lower(), expected_answer.lower()).ratio()

if similarity_ratio > 0.8:
    print('Ответ верный')
else:
    print('Ответ неверный')
    

Вопрос: Для чего в классе используется атрибут slots?


Ответ:  Атрибут __slots__ в классе Python используется для оптимизации памяти и ускорения работы с объектами класса. Он позволяет явно указать, какие атрибуты объекта будут использоваться, а какие нет.


Ответ неверный
